In [1]:
import multiprocess as mp
import tqdm
import geopandas as gpd
import pandas as pd
import shapely
import fiona
shapely.speedups.enable()

### To do
- reduce coordinate precision.


Load in the isochrones for the region

In [2]:
df = gpd.read_file('isochrones/north_west.geojson')

In [3]:
df

,iso_id,id,min,max,center,geometry
0,1,1,0.0,240.0,120.0,"MULTIPOLYGON (((-3.63037 57.16065, -3.88678 57..."
1,2,1,0.0,240.0,120.0,"MULTIPOLYGON (((-3.63019 57.14715, -3.87742 57..."
2,3,1,0.0,240.0,120.0,"MULTIPOLYGON (((-3.61451 57.14990, -3.86663 57..."
3,4,1,0.0,240.0,120.0,"MULTIPOLYGON (((-3.63000 57.13072, -3.86267 57..."
4,5,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.27860 56.77070, -2.18040 56..."
...,...,...,...,...,...,...
9153,9154,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.63156 55.49185, -2.95372 55..."
9154,9155,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.61663 55.48228, -2.93878 55..."
9155,9156,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.73567 55.48873, -3.05782 55..."
9156,9157,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.70590 55.48450, -3.02805 55..."


Load in the centroids from the population grid

In [4]:
centroids = gpd.read_file('data/gb_centroid.geojson')

In [5]:
centroids

,grid_id,population,geometry
0,1,24,POINT (-4.44691 53.42219)
1,2,15,POINT (-4.43188 53.42250)
2,3,10,POINT (-4.38678 53.42341)
3,4,32,POINT (-4.49149 53.41228)
4,5,121,POINT (-4.47646 53.41259)
...,...,...,...
117804,117805,7,POINT (-4.90856 54.66161)
117805,117806,0,POINT (-4.89308 54.66198)
117806,117807,2,POINT (-4.89244 54.65300)
117807,117808,8,POINT (-4.89180 54.64403)


Calculate the total population of the UK

In [6]:
totalpop=centroids['population'].sum()
totalpop

60553399

In [7]:
# define a function that takes a isochrone, filters the centroids within the isochrone and then adds the population.
def inthegoldentriangle(isochrone):
    within=centroids.loc[centroids.within(isochrone)]
    if(within['population'].sum()/totalpop>0.9):
        return True
    else:
        return False    

In [53]:
# test it
inthegoldentriangle2(df['geometry'][4])

False

In [8]:
# run the function across all the isochrones in parallel, save it to a list called output
pool=mp.Pool(8)
output = list(tqdm.tqdm_notebook(pool.imap(inthegoldentriangle, df['geometry']), total=len(df['geometry'])))
pool.close()
pool.join()

<ipython-input-8-50d0e42b5745>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  output = list(tqdm.tqdm_notebook(pool.imap(inthegoldentriangle, df['geometry']), total=len(df['geometry'])))


  0%|          | 0/9158 [00:00<?, ?it/s]

In [59]:
output

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [9]:
# find out how many squares are within the golden triangle
sum(x==True for x in output)

0

In [10]:
# Add the column to the isochrones
df['inside']=output

In [11]:
# Here's all the isochrones that are within the golden triangle
df[df['inside']==True]

,iso_id,id,min,max,center,geometry,inside


In [66]:
# use Henry's lookup
lookup=gpd.read_file('lookups/west_midlands_lookup.geojson')
lookup['inside']=output
lookup

,iso_id,id,min,max,center,population,lon,lat,geometry,inside
0,1,1,0.0,240.0,120.0,77,-1.949058,53.204560,"MULTIPOLYGON (((-2.75444 55.53544, -3.05219 55...",False
1,2,1,0.0,240.0,120.0,72,-1.934086,53.204553,"MULTIPOLYGON (((-2.73947 55.53005, -3.02772 55...",False
2,3,1,0.0,240.0,120.0,242,-1.993973,53.195582,"MULTIPOLYGON (((-3.12151 55.52005, -3.44367 55...",False
3,4,1,0.0,240.0,120.0,43,-2.053834,53.186581,"MULTIPOLYGON (((-3.18137 55.52108, -3.50353 55...",False
4,5,1,0.0,240.0,120.0,117,-1.874254,53.186527,"MULTIPOLYGON (((-1.71318 55.47411, -1.93273 55...",False
...,...,...,...,...,...,...,...,...,...,...
9622,9623,1,0.0,240.0,120.0,69,-2.676405,51.845152,"MULTIPOLYGON (((-0.90456 54.23704, -1.22671 54...",False
9623,9624,1,0.0,240.0,120.0,5,-2.661890,51.845234,"MULTIPOLYGON (((-0.89004 54.17918, -1.21220 54...",False
9624,9625,1,0.0,240.0,120.0,27,-2.647375,51.845315,"MULTIPOLYGON (((-0.87553 54.21896, -1.19768 54...",False
9625,9626,1,0.0,240.0,120.0,0,-2.632860,51.845394,"MULTIPOLYGON (((-3.08654 54.26495, -3.10221 54...",False


In [76]:
# find the centre points of the isochrones
goldentriangle=lookup[lookup['inside']==True]
goldentriangle.to_file('westmidlands-inside.geojson',driver='GeoJSON')

In [68]:
# make a geodataframe with the points
gdf = gpd.GeoDataFrame(goldentriangle, geometry=gpd.points_from_xy(goldentriangle.lon,goldentriangle.lat))
gdf

,iso_id,id,min,max,center,population,lon,lat,geometry,inside
2555,2556,1,0.0,240.0,120.0,28,-2.186494,52.719006,POINT (-2.18649 52.71901),True
2637,2638,1,0.0,240.0,120.0,9,-2.127250,52.710095,POINT (-2.12725 52.71010),True
2638,2639,1,0.0,240.0,120.0,217,-2.112448,52.710110,POINT (-2.11245 52.71011),True
2639,2640,1,0.0,240.0,120.0,154,-2.097647,52.710123,POINT (-2.09765 52.71012),True
2640,2641,1,0.0,240.0,120.0,85,-2.053243,52.710152,POINT (-2.05324 52.71015),True
...,...,...,...,...,...,...,...,...,...,...
6442,6443,1,0.0,240.0,120.0,20,-1.729990,52.323290,POINT (-1.72999 52.32329),True
6526,6527,1,0.0,240.0,120.0,68,-1.862066,52.314527,POINT (-1.86207 52.31453),True
6532,6533,1,0.0,240.0,120.0,21,-1.774051,52.314392,POINT (-1.77405 52.31439),True
6627,6628,1,0.0,240.0,120.0,132,-1.774097,52.305402,POINT (-1.77410 52.30540),True


In [71]:
# make a convex hull around the points, convert it to a geodataframe, and then save it
triangle=gdf.unary_union.convex_hull
hull = gpd.GeoDataFrame(geometry=gpd.GeoSeries(triangle))
hull
hull.set_crs('wgs84')
hull.to_file('goldentriangle-westmidlands.geojson',driver='GeoJSON')

In [75]:
gdf.to_file('goldenpoints-westmidlands.geojson',driver='GeoJSON')

In [ ]:
# testing out the centroid loop
population=0
for index,row in centroids.iterrows():
    if(row['geometry'].within(df['geometry'][0])):
        population=population+row['population']
# if(population/totalpop>0.9):
print(population/totalpop)

In [ ]:
# testing out the isochrone loop

for index,isochrone in df[0:2].iterrows():
    population=0
    for i,centroid in centroids.iterrows():
        if(centroid['geometry'].within(isochrone['geometry'])):
            population=population+centroid['population']
    print(index)
    if(population/totalpop>0.9):
        df['inside']=True
    else:
        df['inside']=False